In [1]:
%load_ext autoreload
%autoreload 2

import numpy
import pandas

import murefi

## Input
The input file is just a table of `Replicate`-ID vs. parameters. The first column name is not important.

In [2]:
df = pandas.read_csv(r"..\tests\testfiles\ParTest.csv", sep=';', index_col=0)

In [3]:
df

,S_0,X_0,mue_max,K_S,Y_XS,t_lag,t_acc
id,,,,,,,
A01,test1A,test1B,3,4,5,6,7
B02,11,test1B,test2C,test2D,15,16,17


## Instantiation
The constructor of the `ParameterMapping` takes override-dictionaries for `bounds` and initial `guesses`.
These dictionaries should contain bounds & guesses for all model dimensions.

To simplify the definition of `bounds` & `guesses`, we could add them as dictionary-properties to the model class.
For demo purposes, this is shown with a mockup model:

In [4]:
class Model:
    def __init__(self):
        self.bounds = dict(
            S_0=(1,2),
            X_0=(3,4),
            mue_max=(5,6),
            K_S=(7,8),
            t_lag=(9,10),
            t_acc=(11,12)
        )
        self.initial_guesses = dict(
            S_0=0.1,
            X_0=0.2,
            mue_max=0.3,
            K_S=0.4,
            t_lag=0.5,
            t_acc=0.6
        )
        return
    
model = Model()

The `bounds` and `initial_guesses` from the model can be exploded (or passed directly).
To override bounds or guesses for a particular model dimension, one can use some dict-magic:

In [5]:
pm = murefi.ParameterMapping(df,
    bounds=dict(
        **model.bounds,
        test2C=(5.1,6.1)
    ),
    guesses={
        **model.initial_guesses,
        **dict(test1A=0.11, X_0=0.21)    # this dictionary overrides entries from the one above
    }
)

### Properties of the `ParameterMapping`
The object exposes several properties that are useful in various contexts.
This includes:
+ `order`: names of model parameters (matches the column order from the DataFrame)
+ `parameters`: dictionary of all unique parameters in the mapping and their corresponding model parameter name
+ `bounds` and `guesses` with `ndim` entries for all the `parameters`
+ `mapping` is a dictionary indicating the names or values of parameters that will end up in replicate-wise parameter vectors

In [6]:
pm.order

('X_0', 'mue_max', 'K_S', 'Y_XS', 't_lag', 't_acc')

In [7]:
pm.ndim

3

In [8]:
pm.parameters

OrderedDict([('test1B', 'X_0'), ('test2C', 'mue_max'), ('test2D', 'K_S')])

In [9]:
pm.mapping

{'test1A': ('test1B', 3.0, 4.0, 5.0, 6.0, 7.0),
 '11': ('test1B', 'test2C', 'test2D', 15.0, 16.0, 17.0)}

In [10]:
pm.bounds

((3, 4), (5.1, 6.1), (7, 8))

In [11]:
pm.guesses

(0.21, 0.3, 0.4)

### Mapping method
The `repmap` method is the most important feature - it maps a global parameter vector (same order as `parameters`) to `Replicate`-wise parameter vectors.

In [12]:
pm.repmap([1,2,13,14])

{'test1A': array([1., 3., 4., 5., 6., 7.]),
 '11': array([ 1.,  2., 13., 15., 16., 17.])}

In [13]:
pm.guesses

(0.21, 0.3, 0.4)